Running cellchat for visiumHD

In [1]:
suppressPackageStartupMessages({
    library(tidyverse)
    library(zellkonverter)
    # library(scater)
    # library(scran)
    # library(scuttle)
    library(SingleCellExperiment)
    library(CellChat)
})

load in adata

In [2]:
sce = readH5AD("tmp/adata_gated.h5ad")

In [3]:
data.input = assay(sce, "X") # X are the log norm counts here, see part 1
meta = data.frame(labels = sce$Subtype_gate,
                  slices = sce$batch,
                  row.names = colnames(sce))
# drop na from meta
meta = meta[!is.na(meta$labels),]
data.input = data.input[,rownames(meta)]
meta$labels = droplevels(meta$labels, exclude = setdiff(levels(meta$labels),unique(meta$labels)))
spatial.locs = reducedDim(sce, 'X_spatial') |> as.data.frame()
scale.factors = list(spot.diameter = 5, spot = 5)
spatial.factors = data.frame(ratio = 1, tol = 5)

Create cellchat object

In [ ]:
cellchat <-
  createCellChat(
    object = data.input,
    meta = meta,
    group.by = "labels",
    datatype = "spatial",
    coordinates = spatial.locs,
    #scale.factors = scale.factors
    spatial.factors = spatial.factors
  )

In [5]:
CellChatDB <- CellChatDB.mouse # use CellChatDB.human if running on human data

CellChatDB.use <- CellChatDB # simply use the default CellChatDB

# set the used database in the object
cellchat@DB <- CellChatDB.use

In [6]:
# subset the expression data of signaling genes for saving computation cost
cellchat <- subsetData(cellchat) # This step is necessary even if using the whole database


In [ ]:
# future::plan("multisession", workers = 8) # do parallel
cellchat <- identifyOverExpressedGenes(cellchat)
cellchat <- identifyOverExpressedInteractions(cellchat)

This part takes a long time to run!
(~3h)

In [ ]:
#future::plan("multisession", workers = 32)
# Typically, contact.range = 10, which is a typical human cell size
cellchat <- computeCommunProb(cellchat,
  type = "thresholdedMean", trim = 0.001,
  distance.use = TRUE, interaction.range = 150,
  contact.range = 50,
  scale.distance = 1
)

In [9]:

# Filter out the cell-cell communication if there are only few number of cells in certain cell groups
cellchat <- filterCommunication(cellchat, min.cells = 10)


In [10]:
cellchat <- computeCommunProbPathway(cellchat)
cellchat <- aggregateNet(cellchat)

Save out the final object

In [11]:
saveRDS(cellchat, file = "tmp/cellchat.rds")